In [ ]:
#!/usr/bin/env python
# coding: utf-8

In[11]:

In [ ]:
get_ipython().magic('pylab inline')

Explore LarterBreakspear model.<br>
<br>
``Run time``: 20 min (workstation circa 2012 Intel Xeon W3520@2.67Ghz)<br>
<br>
``Memory requirement``: ~300 MB<br>
``Storage requirement``: ~150MB<br>
<br>
NOTE: stats were made for a simulation using the 998 region Hagmann<br>
connectivity matrix.

In[12]:

Third party python libraries

In [ ]:
import numpy

Try and import from "The Virtual Brain"

In [ ]:
from tvb.simulator.lab import *
LOG = get_logger('demo')
from tvb.datatypes.time_series import TimeSeriesRegion
import tvb.analyzers.fmri_balloon as bold
from tvb.simulator.plot import timeseries_interactive as timeseries_interactive

####Perform the simulation

In[13]:

In [ ]:
LOG.info("Configuring...")

In[14]:

nitialise a Model, Coupling, and Connectivity.

In [ ]:
lb = models.LarterBreakspear(QV_max=1.0, QZ_max=1.0, 
                             d_V=0.65, d_Z=0.65, 
                             aee=0.36, ani=0.4, ane=1.0, C=0.1)

In [ ]:
lb.variables_of_interest = ["V", "W", "Z"]

In [ ]:
white_matter = connectivity.Connectivity(load_default=True)
white_matter.speed = numpy.array([7.0])

In [ ]:
white_matter_coupling = coupling.HyperbolicTangent(a=0.5*lb.QV_max, 
                                                   midpoint=lb.VT, 
                                                   sigma=lb.d_V)

In[15]:

nitialise an Integrator

In [ ]:
heunint = integrators.HeunDeterministic(dt=0.2)

In[16]:

nitialise some Monitors with period in physical time

In [ ]:
mon_tavg =  monitors.TemporalAverage(period=2.)
mon_bold  = monitors.Bold(period=2000.)
#Bundle them
what_to_watch = (mon_bold, mon_tavg)

In[17]:

nitialise a Simulator -- Model, Connectivity, Integrator, and Monitors.

In [ ]:
sim = simulator.Simulator(model = lb, 
                          connectivity = white_matter,
                          coupling = white_matter_coupling, 
                          integrator = heunint, 
                          monitors = what_to_watch)

In [ ]:
sim.configure()

In[ ]:

In [ ]:
LOG.info("Starting simulation...")
#Perform the simulation
bold_data, bold_time = [], []
tavg_data, tavg_time = [], []

In [ ]:
for raw, tavg in sim(simulation_length=480000):
    if not raw is None:
        bold_time.append(raw[0])
        bold_data.append(raw[1])
    
    if not tavg is None:
        tavg_time.append(tavg[0])
        tavg_data.append(tavg[1])
        
LOG.info("Finished simulation.")

####Plot pretty pictures of what we just did 

In[ ]:

ake the lists numpy.arrays for easier use.

In [ ]:
LOG.info("Converting result to array...")
TAVG_TIME = numpy.array(tavg_time)
BOLD_TIME = numpy.array(bold_time)
BOLD = numpy.array(bold_data)
TAVG = numpy.array(tavg_data)

reate TimeSeries instance

In [ ]:
tsr = TimeSeriesRegion(data = TAVG,
                       time = TAVG_TIME,
                       sample_period = 2.)
tsr.configure()

reate and run the monitor/analyser

In [ ]:
bold_model = bold.BalloonModel(time_series = tsr)
bold_data  = bold_model.evaluate()

In [ ]:
bold_tsr = TimeSeriesRegion(connectivity = white_matter,
                            data = bold_data.data, 
                            time = bold_data.time)

rutty puctures...

In [ ]:
tsi = timeseries_interactive.TimeSeriesInteractive(time_series = bold_tsr)
tsi.configure()
tsi.show()